# Silver Layer Implementation - NYC Yellow Taxi Data Pipeline

## Project Overview
This notebook implements the **Silver Layer** of the Medallion Architecture for NYC Yellow Taxi Trip data. The Silver layer focuses on **data cleaning, validation, and business rule enforcement**.

### Medallion Architecture - Silver Layer Goals
The Silver layer serves as the **cleaned and validated data zone** where we:
1. Load Bronze layer data with quality flags
2. Perform comprehensive data quality analysis
3. Apply business rule validations
4. Handle missing and invalid values
5. Create clean, trustworthy datasets for analytics
6. Document all cleaning decisions with justification

### Dataset Context
- **Source**: Bronze layer output (79.5M records, 24 monthly files)
- **Input Location**: `/home/ubuntu/dat535-2025-group10/bronze_layer/`
- **Quality Baseline**: 89.65% clean, 10.35% flagged in Bronze
- **Environment**: Single VM with 4 vCPUs, 8GB RAM, 40GB storage

### Silver Layer Approach
Following **Part 2 of project.md**, we will:
- Identify cleaning/preprocessing steps through data profiling
- **Implement cleaning using basic MapReduce routines in Spark**
- Profile and tune the implementation for performance
- Result: Clean dataset ready for Gold layer analytics

### Critical Constraint from project.md Part 2:
**"Implement cleaning/preprocessing steps using basic MapReduce routines in Spark (no SQL, Dataframes or similar libraries)"**

This means we must use RDD operations (map, flatMap, filter, reduce, reduceByKey, etc.) for all data cleaning logic, similar to the Bronze layer approach.

### Learning Objectives
- Systematic data quality assessment using MapReduce
- Business rule validation with RDD transformations
- Missing value handling using map/filter operations
- Outlier detection with reduce operations
- Performance optimization for RDD-based cleaning

## Part 1: Environment Setup and Bronze Layer Loading

We'll start by:
1. Importing necessary libraries
2. Initializing SparkSession with appropriate configurations
3. Loading Bronze layer data
4. Understanding the current state of data quality

In [5]:
# Import Required Libraries
import os
import time
from datetime import datetime
from typing import Dict, Any, List, Tuple
import findspark

# Initialize findspark to locate Spark installation
findspark.init()

# PySpark imports - We'll use RDD operations for Silver layer cleaning
from pyspark.sql import SparkSession
from pyspark import SparkContext

print("✓ Libraries imported successfully!")
print(f"Current timestamp: {datetime.now()}")
print(f"Working directory: {os.getcwd()}")
print()
print("NOTE: Silver layer will use RDD operations (MapReduce) for data cleaning")
print("      per Part 2 requirements: 'no SQL, Dataframes or similar libraries'")

✓ Libraries imported successfully!
Current timestamp: 2025-11-22 10:18:11.511455
Working directory: /home/ubuntu/project2

NOTE: Silver layer will use RDD operations (MapReduce) for data cleaning
      per Part 2 requirements: 'no SQL, Dataframes or similar libraries'


In [6]:
# Initialize SparkSession optimized for RDD-based data cleaning
spark = SparkSession.builder \
    .appName("Silver-Layer-RDD-Data-Cleaning") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .getOrCreate()

# Set log level to reduce verbose output
spark.sparkContext.setLogLevel("WARN")

# Get SparkContext for RDD operations
sc = spark.sparkContext

print("=" * 80)
print("SPARK SESSION INITIALIZED FOR SILVER LAYER (RDD-BASED CLEANING)")
print("=" * 80)
print(f"Application Name: {sc.appName}")
print(f"Spark Version: {spark.version}")
print(f"Master: {sc.master}")
print(f"Default Parallelism: {sc.defaultParallelism}")
print(f"Driver Memory: {spark.conf.get('spark.driver.memory')}")
print(f"Executor Memory: {spark.conf.get('spark.executor.memory')}")
print(f"Adaptive Execution: {spark.conf.get('spark.sql.adaptive.enabled')}")
print(f"Shuffle Partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")
print()
print("Processing Mode: RDD MapReduce operations (per Part 2 requirements)")
print("=" * 80)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/22 10:18:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/22 10:18:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SPARK SESSION INITIALIZED FOR SILVER LAYER (RDD-BASED CLEANING)
Application Name: Silver-Layer-RDD-Data-Cleaning
Spark Version: 3.5.0
Master: local[*]
Default Parallelism: 8
Driver Memory: 4g
Executor Memory: 2g
Adaptive Execution: true
Shuffle Partitions: 8

Processing Mode: RDD MapReduce operations (per Part 2 requirements)
Driver Memory: 4g
Executor Memory: 2g
Adaptive Execution: true
Shuffle Partitions: 8

Processing Mode: RDD MapReduce operations (per Part 2 requirements)


### Reflection on Spark Initialization

**What we accomplished:**
- Successfully created SparkSession with application name "Silver-Layer-RDD-Data-Cleaning"
- Using Spark 3.5.0 on local[*] mode (utilizing all 4 vCPUs)
- Memory configuration: 4GB driver, 2GB executor (same as Bronze layer)
- Obtained SparkContext reference for RDD operations
- Optimized settings for RDD-based data cleaning:
  - Default parallelism: 8 partitions (2x vCPUs for optimal CPU utilization)
  - Same configuration as Bronze layer for consistency

**Configuration rationale:**
- Same memory footprint as Bronze layer since we're processing the same 79.5M records
- RDD operations will use default parallelism of 8 for distributed processing
- All cleaning logic will use MapReduce paradigm (map, filter, flatMap, reduce, etc.)

**Next step:**
Load the Bronze layer data using DataFrame (for efficient Parquet reading), then immediately convert to RDD for all cleaning operations.

## Part 1.5: Schema Normalization

The Bronze layer contains schema inconsistencies that must be resolved:
- **Field name variations:** `Airport_fee` vs `airport_fee` (capitalization differences)
- **Data type inconsistencies:** `passenger_count` and `ratecodeid` appear as both DoubleType and LongType

Solution: Create unified schema and normalization MAP function to standardize all records.

### Schema Normalization Implementation

**Objective:**
Create unified schema and normalization MAP function to standardize all Bronze records.

**Schema Inconsistencies Identified:**

**1. Field Name Variations:**
- `Airport_fee` (capital A) vs `airport_fee` (lowercase) - needs standardization

**2. Data Type Variations:**
- `passenger_count`: DoubleType vs LongType across different months
- `ratecodeid`: DoubleType vs LongType across different months

**Solution:**
Define a unified target schema and create RDD-based `normalize_record()` function to:
- Standardize all field names (lowercase)
- Convert all numeric fields to Double for consistency
- Handle any null/missing values appropriately

**Implementation:**
The normalization function is a **pure MAP transformation** - no side effects, just record transformation.

In [7]:
# Define Unified Silver Layer Schema and Normalization Functions
print("=" * 80)
print("DEFINING UNIFIED SILVER LAYER SCHEMA")
print("=" * 80)
print()

# Define target schema - all field names lowercase, consistent types
SILVER_SCHEMA = {
    # Original taxi trip fields (19 fields)
    'vendorid': 'long',
    'tpep_pickup_datetime': 'timestamp',
    'tpep_dropoff_datetime': 'timestamp',
    'passenger_count': 'double',  # Normalize to double
    'trip_distance': 'double',
    'ratecodeid': 'double',  # Normalize to double
    'store_and_fwd_flag': 'string',
    'pulocationid': 'long',
    'dolocationid': 'long',
    'payment_type': 'long',
    'fare_amount': 'double',
    'extra': 'double',
    'mta_tax': 'double',
    'tip_amount': 'double',
    'tolls_amount': 'double',
    'improvement_surcharge': 'double',
    'total_amount': 'double',
    'congestion_surcharge': 'double',
    'airport_fee': 'double',  # Standardize to lowercase
    
    # Bronze metadata fields (5 fields) - keep as-is
    '_bronze_ingestion_timestamp': 'string',
    '_bronze_source_file': 'string',
    '_bronze_record_id': 'string',
    '_bronze_status': 'string',
    '_bronze_quality_flags': 'string'
}

print("Target Schema Defined:")
print(f"  Total fields: {len(SILVER_SCHEMA)}")
print(f"  Original taxi fields: 19")
print(f"  Bronze metadata fields: 5")
print()

# Field name mapping for variations
FIELD_NAME_MAPPING = {
    'Airport_fee': 'airport_fee',  # Capital A -> lowercase
    'RatecodeID': 'ratecodeid',    # Just in case
    'PULocationID': 'pulocationid',  # Just in case
    'DOLocationID': 'dolocationid',  # Just in case
    'VendorID': 'vendorid'  # Just in case
}

print("Field Name Normalization Rules:")
for old_name, new_name in FIELD_NAME_MAPPING.items():
    print(f"  {old_name:20s} -> {new_name}")
print()

# RDD-based normalization function
def normalize_record(record: Dict) -> Dict:
    """
    MAP function: Normalize a single record to match Silver schema.
    
    Handles:
    1. Field name standardization (all lowercase)
    2. Data type conversion (passenger_count, ratecodeid to double)
    3. Null value handling
    
    Args:
        record: Dictionary representing a Bronze layer record
    
    Returns:
        Normalized dictionary matching Silver schema
    """
    normalized = {}
    
    # First pass: normalize all field names to lowercase
    for key, value in record.items():
        # Check if field needs explicit mapping
        if key in FIELD_NAME_MAPPING:
            normalized_key = FIELD_NAME_MAPPING[key]
        else:
            # Default: convert to lowercase
            normalized_key = key.lower()
        
        normalized[normalized_key] = value
    
    # Second pass: ensure data type consistency for problematic fields
    # Convert passenger_count to double if it exists
    if 'passenger_count' in normalized and normalized['passenger_count'] is not None:
        try:
            normalized['passenger_count'] = float(normalized['passenger_count'])
        except (ValueError, TypeError):
            normalized['passenger_count'] = None
    
    # Convert ratecodeid to double if it exists
    if 'ratecodeid' in normalized and normalized['ratecodeid'] is not None:
        try:
            normalized['ratecodeid'] = float(normalized['ratecodeid'])
        except (ValueError, TypeError):
            normalized['ratecodeid'] = None
    
    return normalized

print("=" * 80)
print("NORMALIZATION FUNCTION DEFINED")
print("=" * 80)
print("Function: normalize_record()")
print("  • Standardizes field names to lowercase")
print("  • Converts passenger_count to double")
print("  • Converts ratecodeid to double")
print("  • Handles null values gracefully")
print()
print("This is a MAP operation for RDD-based cleaning!")
print("=" * 80)

DEFINING UNIFIED SILVER LAYER SCHEMA

Target Schema Defined:
  Total fields: 24
  Original taxi fields: 19
  Bronze metadata fields: 5

Field Name Normalization Rules:
  Airport_fee          -> airport_fee
  RatecodeID           -> ratecodeid
  PULocationID         -> pulocationid
  DOLocationID         -> dolocationid
  VendorID             -> vendorid

NORMALIZATION FUNCTION DEFINED
Function: normalize_record()
  • Standardizes field names to lowercase
  • Converts passenger_count to double
  • Converts ratecodeid to double
  • Handles null values gracefully

This is a MAP operation for RDD-based cleaning!


### Reflection on Schema Normalization

**What we accomplished:**
Successfully created RDD-based schema normalization function that standardizes heterogeneous Bronze schemas.

**Key Normalizations:**
- `Airport_fee` → `airport_fee`
- `VendorID` → `vendorid`
- `PULocationID` → `pulocationid`
- `DOLocationID` → `dolocationid`
- `RatecodeID` → `ratecodeid`
- `passenger_count`: converted to float for consistency

**MapReduce Operation:**
The `normalize_record()` function is a **MAP transformation** that takes each record (dictionary) and returns a normalized version. This is pure functional programming - no side effects, just transformation.

**Next Step:**
Implement data enrichment to add human-readable fields for ID lookups, followed by quality validation.

## Part 2: Data Quality Validation with RDD Operations

Now we'll implement comprehensive data cleaning using MapReduce operations:
1. **Null value analysis** - Identify missing data patterns
2. **Business rule validation** - Check fare amounts, trip distances, timestamps
3. **Quality flag generation** - Mark records needing attention
4. **Statistical profiling** - Understand data distributions

All using RDD operations: map, filter, flatMap, reduce, reduceByKey

In [8]:
# Define Data Quality Validation Functions (RDD MapReduce)

def validate_record_quality(record: Dict) -> Tuple[Dict, List[str]]:
    """
    MAP function: Validate a single record against business rules.
    
    Returns:
        Tuple of (record, list_of_quality_issues)
    """
    issues = []
    
    # 1. NULL VALUE CHECKS
    critical_fields = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                      'trip_distance', 'total_amount']
    for field in critical_fields:
        if record.get(field) is None:
            issues.append(f'null_{field}')
    
    # 2. BUSINESS LOGIC VALIDATIONS
    
    # Check: pickup before dropoff
    pickup = record.get('tpep_pickup_datetime')
    dropoff = record.get('tpep_dropoff_datetime')
    if pickup and dropoff:
        if dropoff <= pickup:
            issues.append('invalid_trip_duration')
    
    # Check: trip distance positive
    distance = record.get('trip_distance')
    if distance is not None:
        if distance <= 0:
            issues.append('invalid_trip_distance')
        elif distance > 100:  # Suspiciously long trip (>100 miles in NYC)
            issues.append('suspicious_trip_distance')
    
    # Check: passenger count reasonable
    passengers = record.get('passenger_count')
    if passengers is not None:
        if passengers <= 0:
            issues.append('invalid_passenger_count')
        elif passengers > 6:  # NYC taxis typically max 4-5, but allow up to 6
            issues.append('suspicious_passenger_count')
    
    # Check: fare amounts non-negative
    fare_fields = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 
                   'tolls_amount', 'improvement_surcharge', 'total_amount',
                   'congestion_surcharge', 'airport_fee']
    
    for field in fare_fields:
        value = record.get(field)
        if value is not None and value < 0:
            issues.append(f'negative_{field}')
    
    # Check: total amount consistency (basic check)
    total = record.get('total_amount')
    fare = record.get('fare_amount')
    if total is not None and fare is not None:
        if total > 0 and fare == 0:
            issues.append('suspicious_fare_structure')
        elif total > 1000:  # Very expensive ride
            issues.append('suspicious_total_amount')
    
    # Check: location IDs present
    if record.get('pulocationid') is None or record.get('dolocationid') is None:
        issues.append('missing_location_info')
    
    return (record, issues)


def extract_quality_issues(validation_result: Tuple[Dict, List[str]]) -> List[Tuple[str, int]]:
    """
    FLATMAP function: Extract individual quality issues from validation result.
    
    Returns:
        List of (issue_name, count=1) tuples for aggregation
    """
    record, issues = validation_result
    if issues:
        return [(issue, 1) for issue in issues]
    else:
        return [('no_issues', 1)]


print("=" * 80)
print("DATA QUALITY VALIDATION FUNCTIONS DEFINED")
print("=" * 80)
print()
print("Functions for RDD MapReduce pipeline:")
print("-" * 80)
print("1. validate_record_quality()")
print("   • MAP operation")
print("   • Returns: (record, list_of_issues)")
print("   • Checks:")
print("     - Null values in critical fields")
print("     - Pickup before dropoff datetime")
print("     - Positive trip distances")
print("     - Reasonable passenger counts")
print("     - Non-negative fare amounts")
print("     - Total amount consistency")
print("     - Location information present")
print()
print("2. extract_quality_issues()")
print("   • FLATMAP operation") 
print("   • Extracts individual issues for counting")
print("   • Returns: [(issue_name, 1), ...]")
print()
print("=" * 80)

DATA QUALITY VALIDATION FUNCTIONS DEFINED

Functions for RDD MapReduce pipeline:
--------------------------------------------------------------------------------
1. validate_record_quality()
   • MAP operation
   • Returns: (record, list_of_issues)
   • Checks:
     - Null values in critical fields
     - Pickup before dropoff datetime
     - Positive trip distances
     - Reasonable passenger counts
     - Non-negative fare amounts
     - Total amount consistency
     - Location information present

2. extract_quality_issues()
   • FLATMAP operation
   • Extracts individual issues for counting
   • Returns: [(issue_name, 1), ...]



### Reflection on Quality Validation

**What we accomplished:**
Successfully implemented comprehensive data quality validation using pure RDD MapReduce operations!

**Quality Validation Rules (14 total):**

**Critical Issues (Records Removed):**
1. Null critical timestamps or amounts
2. Invalid trip duration (dropoff before pickup)
3. Invalid trip distance (≤ 0 miles)
4. Invalid passenger count (≤ 0 passengers)

**Non-Critical Issues (Records Flagged):**
- Negative fare amounts (might be refunds/corrections)
- Suspicious values (outliers)
- Missing location information

**MapReduce Pipeline:**
```
Raw Data → map(normalize) → map(enrich) → map(validate) → filter(!remove) → map(add_metadata)
```

All operations are RDD-based transformations, following Part 2 requirements perfectly!

**Next Step:**
Process all 24 files using the complete pipeline including enrichment.

## Part 3: Data Cleaning Strategy and Implementation

Based on quality analysis, we'll implement a cleaning strategy using RDD filter operations:

**Cleaning Approach:**
1. **Remove records with critical issues:**
   - Invalid trip duration (dropoff before pickup)
   - Zero/negative trip distance
   - Zero/negative passenger count
   - Missing critical timestamps

2. **Keep but flag suspicious records:**
   - Negative fare amounts (might be refunds/corrections)
   - Very high amounts or distances (outliers but potentially valid)
   
3. **Add Silver layer metadata:**
   - Cleaning timestamp
   - Issues detected
   - Clean/flagged status

This follows Medallion Architecture: Bronze preserves all, Silver cleans and validates.

In [9]:
# Define Silver Layer Cleaning Functions

def should_remove_record(issues: List[str]) -> bool:
    """
    FILTER function: Determine if record should be removed (True) or kept (False).
    
    Remove records with critical data quality issues that make them unusable.
    """
    critical_issues = {
        'null_tpep_pickup_datetime',
        'null_tpep_dropoff_datetime',
        'null_trip_distance',
        'null_total_amount',
        'invalid_trip_duration',
        'invalid_trip_distance',
        'invalid_passenger_count'
    }
    
    # Remove if any critical issue present
    return any(issue in critical_issues for issue in issues)


def create_silver_record(validation_result: Tuple[Dict, List[str]]) -> Dict:
    """
    MAP function: Create Silver layer record with cleaning metadata.
    
    Adds Silver-specific fields tracking the cleaning process.
    """
    record, issues = validation_result
    
    # Create Silver record (copy of normalized record)
    silver_record = record.copy()
    
    # Add Silver layer metadata
    silver_record['_silver_cleaning_timestamp'] = datetime.now().isoformat()
    silver_record['_silver_quality_issues'] = ','.join(issues) if issues else None
    silver_record['_silver_status'] = 'flagged' if (issues and issues != ['no_issues']) else 'clean'
    
    # Count of issues (excluding 'no_issues')
    issue_count = len([i for i in issues if i != 'no_issues'])
    silver_record['_silver_issue_count'] = issue_count
    
    return silver_record


print("=" * 80)
print("SILVER LAYER CLEANING FUNCTIONS DEFINED")
print("=" * 80)
print()
print("Functions for RDD cleaning pipeline:")
print("-" * 80)
print()
print("1. should_remove_record(issues)")
print("   • FILTER predicate function")
print("   • Returns: True if record should be removed")
print("   • Critical issues that cause removal:")
print("     - Null critical timestamps or amounts")
print("     - Invalid trip duration (dropoff before pickup)")
print("     - Invalid trip distance (≤ 0)")
print("     - Invalid passenger count (≤ 0)")
print()
print("2. create_silver_record(validation_result)")
print("   • MAP transformation")
print("   • Adds Silver metadata:")
print("     - _silver_cleaning_timestamp")
print("     - _silver_quality_issues")
print("     - _silver_status (clean/flagged)")
print("     - _silver_issue_count")
print()
print("=" * 80)
print("Cleaning Strategy:")
print("  • Remove ~2-3% of records with critical issues")
print("  • Keep ~97-98% including flagged records for transparency")
print("  • Silver layer will have 28 fields (24 Bronze + 4 Silver metadata)")
print("=" * 80)

SILVER LAYER CLEANING FUNCTIONS DEFINED

Functions for RDD cleaning pipeline:
--------------------------------------------------------------------------------

1. should_remove_record(issues)
   • FILTER predicate function
   • Returns: True if record should be removed
   • Critical issues that cause removal:
     - Null critical timestamps or amounts
     - Invalid trip duration (dropoff before pickup)
     - Invalid trip distance (≤ 0)
     - Invalid passenger count (≤ 0)

2. create_silver_record(validation_result)
   • MAP transformation
   • Adds Silver metadata:
     - _silver_cleaning_timestamp
     - _silver_quality_issues
     - _silver_status (clean/flagged)
     - _silver_issue_count

Cleaning Strategy:
  • Remove ~2-3% of records with critical issues
  • Keep ~97-98% including flagged records for transparency
  • Silver layer will have 28 fields (24 Bronze + 4 Silver metadata)


### Reflection on Complete Silver Layer Pipeline

**What we accomplished:**
Successfully implemented complete data cleaning AND enrichment pipeline using **exclusively RDD MapReduce operations**!

**Complete Pipeline:**
```
Bronze Data (Parquet)
  ↓ Load & Convert to RDD
  ↓ MAP: normalize_record() ← Schema standardization
  ↓ MAP: enrich_record() ← Add human-readable fields
  ↓ MAP: validate_record_quality() ← Quality validation
  ↓ FILTER: !should_remove_record() ← Remove critical issues
  ↓ MAP: create_silver_record() ← Add Silver metadata
  ↓ Silver Data (Clean + Enriched + Flagged)
```

**Schema Transformation:**
-  All field names standardized to lowercase
-  Data types normalized (passenger_count, ratecodeid → Double)
-  **9 enrichment fields added** (locations, vendor, rate, payment)
-  4 Silver metadata fields added
-  **Final schema: 37 fields** (19 original + 9 enrichment + 5 Bronze + 4 Silver)

**Quality Strategy:**
- **Critical Issues**: Removed (3.15% of records) - data is unusable
- **Non-Critical Issues**: Flagged (1.34% of kept records) - data kept for analysis
- **Clean Records**: 98.66% of kept records have no issues

**Silver Layer Metadata:**
- `_silver_cleaning_timestamp`: When cleaning occurred
- `_silver_quality_issues`: Comma-separated list of issues found
- `_silver_status`: 'clean' or 'flagged'
- `_silver_issue_count`: Number of quality issues

**Part 2 Requirements Met:**
 All cleaning implemented using basic MapReduce routines
 No SQL, DataFrames, or similar libraries used for processing
map(), filter(), flatMap(), reduceByKey() operations demonstrated
 Result converted to DataFrame for output

**Next Step:**
Batch process all 24 files with complete enriched pipeline!

---

## Part 3B: Data Enrichment (Adding Human-Readable Fields)

Before validation, we'll enrich the data by adding human-readable descriptions for ID fields.

**Enrichment Strategy:**
- Load lookup data (taxi zones, vendor names, rate codes, payment types)
- Create enrichment MAP function to add descriptive fields
- Keep original IDs (needed for joins) + add new descriptive fields
- Apply AFTER normalization, BEFORE validation in pipeline

**Fields to Enrich:**
1. **Location IDs** → Borough, Zone, Service Zone (from taxi_zone_lookup.csv)
2. **Vendor ID** → Vendor Name
3. **Rate Code ID** → Rate Description
4. **Payment Type** → Payment Method

This follows best practice: Silver layer = cleaned + enriched data ready for analytics.

In [ ]:
# Load Enrichment Lookup Data
import csv

# 1. Load Taxi Zone Lookup Data
taxi_zone_lookup_path = "/home/ubuntu/dat535-2025-group10/Metadata/taxi_zone_lookup.csv"
taxi_zones = {}

with open(taxi_zone_lookup_path, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        location_id = int(row['LocationID'])
        taxi_zones[location_id] = {
            'borough': row['Borough'],
            'zone': row['Zone'],
            'service_zone': row['service_zone']
        }

print("=" * 80)
print("ENRICHMENT LOOKUP DATA LOADED")
print("=" * 80)
print()
print(f"1. Taxi Zone Lookup: {len(taxi_zones)} zones loaded")
print("   Sample zones:")
for zone_id in [1, 132, 138, 264, 265]:
    if zone_id in taxi_zones:
        info = taxi_zones[zone_id]
        print(f"   • ID {zone_id}: {info['zone']}, {info['borough']} ({info['service_zone']})")
print()

# 2. Vendor ID Mapping (from TLC data dictionary)
vendor_mapping = {
    1: "Creative Mobile Technologies",
    2: "Curb Mobility",
    6: "Myle Technologies",
    7: "Helix"
}
print(f"2. Vendor Mapping: {len(vendor_mapping)} vendors")
for vid, name in vendor_mapping.items():
    print(f"   • {vid}: {name}")
print()

# 3. Rate Code Mapping (from TLC data dictionary)
ratecode_mapping = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
    99: "Unknown"
}
print(f"3. Rate Code Mapping: {len(ratecode_mapping)} rate codes")
for rid, desc in ratecode_mapping.items():
    print(f"   • {rid}: {desc}")
print()

# 4. Payment Type Mapping (from TLC data dictionary)
payment_mapping = {
    0: "Flex Fare",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}
print(f"4. Payment Type Mapping: {len(payment_mapping)} payment types")
for pid, method in payment_mapping.items():
    print(f"   • {pid}: {method}")
print()

print("=" * 80)
print("✓ All lookup data ready for enrichment")
print("=" * 80)

ENRICHMENT LOOKUP DATA LOADED

1. Taxi Zone Lookup: 265 zones loaded
   Sample zones:
   • ID 1: Newark Airport, EWR (EWR)
   • ID 132: JFK Airport, Queens (Airports)
   • ID 138: LaGuardia Airport, Queens (Airports)
   • ID 264: N/A, Unknown (N/A)
   • ID 265: Outside of NYC, N/A (N/A)

2. Vendor Mapping: 4 vendors
   • 1: Creative Mobile Technologies
   • 2: Curb Mobility
   • 6: Myle Technologies
   • 7: Helix

3. Rate Code Mapping: 7 rate codes
   • 1: Standard rate
   • 2: JFK
   • 3: Newark
   • 4: Nassau or Westchester
   • 5: Negotiated fare
   • 6: Group ride
   • 99: Unknown

4. Payment Type Mapping: 7 payment types
   • 0: Flex Fare
   • 1: Credit card
   • 2: Cash
   • 3: No charge
   • 4: Dispute
   • 5: Unknown
   • 6: Voided trip

✓ All lookup data ready for enrichment


In [3]:
# Define Data Enrichment Function

def enrich_record(record):
    """
    MAP operation: Enrich record with human-readable descriptions for ID fields.
    
    Adds 9 new fields:
    - pickup_borough, pickup_zone, pickup_service_zone
    - dropoff_borough, dropoff_zone, dropoff_service_zone
    - vendor_name
    - rate_description
    - payment_method
    
    Keeps original ID fields for joins in Gold layer.
    """
    enriched = record.copy()
    
    # Enrich Pickup Location
    pu_location_id = record.get('pulocationid')
    if pu_location_id and pu_location_id in taxi_zones:
        zone_info = taxi_zones[pu_location_id]
        enriched['pickup_borough'] = zone_info['borough']
        enriched['pickup_zone'] = zone_info['zone']
        enriched['pickup_service_zone'] = zone_info['service_zone']
    else:
        enriched['pickup_borough'] = 'Unknown'
        enriched['pickup_zone'] = 'Unknown'
        enriched['pickup_service_zone'] = 'Unknown'
    
    # Enrich Dropoff Location
    do_location_id = record.get('dolocationid')
    if do_location_id and do_location_id in taxi_zones:
        zone_info = taxi_zones[do_location_id]
        enriched['dropoff_borough'] = zone_info['borough']
        enriched['dropoff_zone'] = zone_info['zone']
        enriched['dropoff_service_zone'] = zone_info['service_zone']
    else:
        enriched['dropoff_borough'] = 'Unknown'
        enriched['dropoff_zone'] = 'Unknown'
        enriched['dropoff_service_zone'] = 'Unknown'
    
    # Enrich Vendor ID
    vendor_id = record.get('vendorid')
    enriched['vendor_name'] = vendor_mapping.get(vendor_id, 'Unknown')
    
    # Enrich Rate Code
    rate_code = record.get('ratecodeid')
    if rate_code is not None:
        enriched['rate_description'] = ratecode_mapping.get(int(rate_code), 'Unknown')
    else:
        enriched['rate_description'] = 'Unknown'
    
    # Enrich Payment Type
    payment_type = record.get('payment_type')
    if payment_type is not None:
        enriched['payment_method'] = payment_mapping.get(int(payment_type), 'Unknown')
    else:
        enriched['payment_method'] = 'Unknown'
    
    return enriched


print("=" * 80)
print("DATA ENRICHMENT FUNCTION DEFINED")
print("=" * 80)
print()
print("Function: enrich_record(record)")
print("  • Type: MAP operation")
print("  • Input: Normalized record dictionary")
print("  • Output: Record with 9 additional enrichment fields")
print()
print("Enrichment Fields Added:")
print("-" * 80)
print("  Location Enrichment (6 fields):")
print("    • pickup_borough, pickup_zone, pickup_service_zone")
print("    • dropoff_borough, dropoff_zone, dropoff_service_zone")
print()
print("  ID Enrichment (3 fields):")
print("    • vendor_name (from vendorid)")
print("    • rate_description (from ratecodeid)")
print("    • payment_method (from payment_type)")
print()
print("Strategy:")
print("  • Keep original ID fields (needed for joins)")
print("  • Add human-readable descriptions alongside IDs")
print("  • Handle missing/unknown IDs gracefully with 'Unknown'")
print()
print("=" * 80)
print("Total Silver Schema: 19 original + 9 enrichment + 5 Bronze + 4 Silver = 37 fields")
print("=" * 80)

DATA ENRICHMENT FUNCTION DEFINED

Function: enrich_record(record)
  • Type: MAP operation
  • Input: Normalized record dictionary
  • Output: Record with 9 additional enrichment fields

Enrichment Fields Added:
--------------------------------------------------------------------------------
  Location Enrichment (6 fields):
    • pickup_borough, pickup_zone, pickup_service_zone
    • dropoff_borough, dropoff_zone, dropoff_service_zone

  ID Enrichment (3 fields):
    • vendor_name (from vendorid)
    • rate_description (from ratecodeid)
    • payment_method (from payment_type)

Strategy:
  • Keep original ID fields (needed for joins)
  • Add human-readable descriptions alongside IDs
  • Handle missing/unknown IDs gracefully with 'Unknown'

Total Silver Schema: 19 original + 9 enrichment + 5 Bronze + 4 Silver = 37 fields


In [ ]:
# Test Enrichment Function on Sample Data
test_file = "/home/ubuntu/dat535-2025-group10/bronze_layer/yellow_tripdata_2024-01_bronze"
df_test = spark.read.parquet(test_file)
rdd_test = df_test.rdd.map(lambda row: row.asDict()).take(3)

print("=" * 80)
print("TESTING ENRICHMENT FUNCTION")
print("=" * 80)
print()

for i, record in enumerate(rdd_test, 1):
    # Normalize first
    normalized = normalize_record(record)
    
    # Then enrich
    enriched = enrich_record(normalized)
    
    print(f"Sample Record {i}:")
    print("-" * 80)
    print(f"  Vendor: ID {enriched.get('vendorid')} → {enriched.get('vendor_name')}")
    print(f"  Rate Code: ID {enriched.get('ratecodeid')} → {enriched.get('rate_description')}")
    print(f"  Payment: ID {enriched.get('payment_type')} → {enriched.get('payment_method')}")
    print()
    print(f"  Pickup: ID {enriched.get('pulocationid')}")
    print(f"    → {enriched.get('pickup_zone')}, {enriched.get('pickup_borough')}")
    print(f"    → Service Zone: {enriched.get('pickup_service_zone')}")
    print()
    print(f"  Dropoff: ID {enriched.get('dolocationid')}")
    print(f"    → {enriched.get('dropoff_zone')}, {enriched.get('dropoff_borough')}")
    print(f"    → Service Zone: {enriched.get('dropoff_service_zone')}")
    print()

print("=" * 80)
print("✓ Enrichment function working correctly!")
print("✓ Ready to integrate into batch processing pipeline")
print("=" * 80)

TESTING ENRICHMENT FUNCTION

Sample Record 1:
--------------------------------------------------------------------------------
  Vendor: ID 2 → Curb Mobility
  Rate Code: ID 1.0 → Standard rate
  Payment: ID 1 → Credit card

  Pickup: ID 48
    → Clinton East, Manhattan
    → Service Zone: Yellow Zone

  Dropoff: ID 68
    → East Chelsea, Manhattan
    → Service Zone: Yellow Zone

Sample Record 2:
--------------------------------------------------------------------------------
  Vendor: ID 2 → Curb Mobility
  Rate Code: ID 1.0 → Standard rate
  Payment: ID 1 → Credit card

  Pickup: ID 231
    → TriBeCa/Civic Center, Manhattan
    → Service Zone: Yellow Zone

  Dropoff: ID 164
    → Midtown South, Manhattan
    → Service Zone: Yellow Zone

Sample Record 3:
--------------------------------------------------------------------------------
  Vendor: ID 2 → Curb Mobility
  Rate Code: ID 1.0 → Standard rate
  Payment: ID 1 → Credit card

  Pickup: ID 90
    → Flatiron, Manhattan
    → Servic

---

## Part 4: Batch Processing with Enrichment

Now we'll reprocess all 24 Bronze files with the complete Silver pipeline:

**Complete RDD Pipeline:**
1. Load Bronze Parquet → RDD of dictionaries
2. **MAP:** `normalize_record()` - Schema standardization
3. **MAP:** `enrich_record()` - Add human-readable fields ← NEW
4. **MAP:** `validate_record_quality()` - Quality checks
5. **FILTER:** `!should_remove_record()` - Remove critical issues
6. **MAP:** `create_silver_record()` - Add Silver metadata
7. Convert to DataFrame and write Parquet

**Updated Schema:** 37 fields total
- 19 original taxi fields
- 9 enrichment fields (locations, vendor, rate, payment)
- 5 Bronze metadata fields
- 4 Silver metadata fields

This follows best practice: Silver = cleaned + enriched data ready for analytics.

In [ ]:
# Batch Processing: Clean All 24 Files
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

bronze_path = "/home/ubuntu/dat535-2025-group10/bronze_layer"
silver_output_path = "/home/ubuntu/dat535-2025-group10/silver_layer"

# Create output directory
if not os.path.exists(silver_output_path):
    os.makedirs(silver_output_path)
    print(f"✓ Created Silver layer output directory: {silver_output_path}")
else:
    print(f"✓ Using existing Silver layer directory: {silver_output_path}")

print()
print("=" * 80)
print("BATCH PROCESSING - ALL 24 FILES")
print("=" * 80)
print(f"Source: {bronze_path}")
print(f"Target: {silver_output_path}")
print()

# Get all Bronze files
bronze_dirs = sorted([d for d in os.listdir(bronze_path) if os.path.isdir(os.path.join(bronze_path, d))])
print(f"Files to process: {len(bronze_dirs)}")
print()

# Define explicit schema for Silver DataFrame (37 fields: 19 original + 9 enrichment + 5 Bronze + 4 Silver)
silver_schema = StructType([
    # Original taxi fields (19)
    StructField("vendorid", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", DoubleType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("ratecodeid", DoubleType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pulocationid", LongType(), True),
    StructField("dolocationid", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True),
    # Enrichment fields (9)
    StructField("pickup_borough", StringType(), True),
    StructField("pickup_zone", StringType(), True),
    StructField("pickup_service_zone", StringType(), True),
    StructField("dropoff_borough", StringType(), True),
    StructField("dropoff_zone", StringType(), True),
    StructField("dropoff_service_zone", StringType(), True),
    StructField("vendor_name", StringType(), True),
    StructField("rate_description", StringType(), True),
    StructField("payment_method", StringType(), True),
    # Bronze metadata (5)
    StructField("_bronze_ingestion_timestamp", StringType(), True),
    StructField("_bronze_source_file", StringType(), True),
    StructField("_bronze_record_id", StringType(), True),
    StructField("_bronze_status", StringType(), True),
    StructField("_bronze_quality_flags", StringType(), True),
    # Silver metadata (4)
    StructField("_silver_cleaning_timestamp", StringType(), True),
    StructField("_silver_quality_issues", StringType(), True),
    StructField("_silver_status", StringType(), True),
    StructField("_silver_issue_count", LongType(), True)
])

# Track overall statistics
overall_stats = {
    'files_processed': 0,
    'total_input_records': 0,
    'total_removed_records': 0,
    'total_output_records': 0,
    'total_clean_records': 0,
    'total_flagged_records': 0,
    'processing_times': [],
    'file_stats': []
}

print("Processing files (RDD MapReduce pipeline):")
print("-" * 80)

# Process each file
for file_idx, bronze_dir in enumerate(bronze_dirs, 1):
    file_start_time = time.time()
    
    # Load Bronze file
    bronze_file_path = os.path.join(bronze_path, bronze_dir)
    df_bronze = spark.read.parquet(bronze_file_path)
    
    # RDD Processing Pipeline (all MapReduce operations)
    rdd_bronze = df_bronze.rdd.map(lambda row: row.asDict())
    rdd_normalized = rdd_bronze.map(normalize_record)
    rdd_enriched = rdd_normalized.map(enrich_record)  # NEW: Add enrichment
    rdd_validated = rdd_enriched.map(validate_record_quality)
    rdd_filtered = rdd_validated.filter(lambda x: not should_remove_record(x[1]))
    rdd_silver = rdd_filtered.map(create_silver_record)
    
    # Count records (trigger computation)
    input_count = rdd_bronze.count()
    output_count = rdd_silver.count()
    removed_count = input_count - output_count
    
    # Get status distribution using REDUCEBYKEY
    status_counts = rdd_silver \
        .map(lambda rec: (rec['_silver_status'], 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .collectAsMap()
    
    clean_count = status_counts.get('clean', 0)
    flagged_count = status_counts.get('flagged', 0)
    
    # Convert dictionary RDD to Row RDD, then to DataFrame with explicit schema
    # MAP operation: Convert dict to tuple in schema field order (37 fields)
    def dict_to_tuple(d):
        return (
            # Original 19 fields
            d.get('vendorid'), d.get('tpep_pickup_datetime'), d.get('tpep_dropoff_datetime'),
            d.get('passenger_count'), d.get('trip_distance'), d.get('ratecodeid'),
            d.get('store_and_fwd_flag'), d.get('pulocationid'), d.get('dolocationid'),
            d.get('payment_type'), d.get('fare_amount'), d.get('extra'),
            d.get('mta_tax'), d.get('tip_amount'), d.get('tolls_amount'),
            d.get('improvement_surcharge'), d.get('total_amount'), d.get('congestion_surcharge'),
            d.get('airport_fee'),
            # Enrichment 9 fields
            d.get('pickup_borough'), d.get('pickup_zone'), d.get('pickup_service_zone'),
            d.get('dropoff_borough'), d.get('dropoff_zone'), d.get('dropoff_service_zone'),
            d.get('vendor_name'), d.get('rate_description'), d.get('payment_method'),
            # Bronze 5 fields
            d.get('_bronze_ingestion_timestamp'), d.get('_bronze_source_file'),
            d.get('_bronze_record_id'), d.get('_bronze_status'), d.get('_bronze_quality_flags'),
            # Silver 4 fields
            d.get('_silver_cleaning_timestamp'), d.get('_silver_quality_issues'),
            d.get('_silver_status'), d.get('_silver_issue_count')
        )
    
    rdd_tuples = rdd_silver.map(dict_to_tuple)
    silver_df = spark.createDataFrame(rdd_tuples, schema=silver_schema)
    
    # Write to Silver layer
    output_file_path = os.path.join(silver_output_path, bronze_dir.replace('_bronze', '_silver'))
    silver_df.write.mode('overwrite').parquet(output_file_path)
    
    # Release memory
    rdd_silver.unpersist()
    del rdd_bronze, rdd_normalized, rdd_enriched, rdd_validated, rdd_filtered, rdd_silver, rdd_tuples, silver_df, df_bronze
    
    # Record statistics
    file_time = time.time() - file_start_time
    overall_stats['files_processed'] += 1
    overall_stats['total_input_records'] += input_count
    overall_stats['total_removed_records'] += removed_count
    overall_stats['total_output_records'] += output_count
    overall_stats['total_clean_records'] += clean_count
    overall_stats['total_flagged_records'] += flagged_count
    overall_stats['processing_times'].append(file_time)
    
    # Store per-file stats
    overall_stats['file_stats'].append({
        'file': bronze_dir,
        'input': input_count,
        'removed': removed_count,
        'output': output_count,
        'clean': clean_count,
        'flagged': flagged_count,
        'time': file_time
    })
    
    # Progress output
    removal_pct = (removed_count / input_count * 100) if input_count > 0 else 0
    clean_pct = (clean_count / output_count * 100) if output_count > 0 else 0
    
    print(f"[{file_idx:2d}/24] {bronze_dir:35s} | "
          f"In: {input_count:7,} | Out: {output_count:7,} | "
          f"Removed: {removal_pct:5.2f}% | Clean: {clean_pct:5.2f}% | "
          f"{file_time:5.1f}s")

print("-" * 80)
print()
print("=" * 80)
print("BATCH PROCESSING COMPLETE!")
print("=" * 80)

✓ Using existing Silver layer directory: /home/ubuntu/project2/silver_layer

BATCH PROCESSING - ALL 24 FILES
Source: /home/ubuntu/project2/bronze_layer
Target: /home/ubuntu/project2/silver_layer

Files to process: 24

Processing files (RDD MapReduce pipeline):
--------------------------------------------------------------------------------


25/11/22 10:22:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/22 10:22:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[ 1/24] yellow_tripdata_2023-01_bronze      | In: 3,066,766 | Out: 2,970,852 | Removed:  3.13% | Clean: 99.24% | 225.1s


[ 2/24] yellow_tripdata_2023-02_bronze      | In: 2,913,955 | Out: 2,826,497 | Removed:  3.00% | Clean: 99.22% | 212.0s


[ 3/24] yellow_tripdata_2023-03_bronze      | In: 3,403,766 | Out: 3,297,701 | Removed:  3.12% | Clean: 99.19% | 247.2s


...

[20/24] yellow_tripdata_2024-08_bronze      | In: 2,979,183 | Out: 2,896,886 | Removed:  2.76% | Clean: 98.12% |  90.3s


[21/24] yellow_tripdata_2024-09_bronze      | In: 3,633,030 | Out: 3,523,197 | Removed:  3.02% | Clean: 98.07% | 105.1s


[22/24] yellow_tripdata_2024-10_bronze      | In: 3,833,771 | Out: 3,717,924 | Removed:  3.02% | Clean: 98.11% | 110.1s


[23/24] yellow_tripdata_2024-11_bronze      | In: 3,646,369 | Out: 3,549,715 | Removed:  2.65% | Clean: 98.11% | 102.3s


[24/24] yellow_tripdata_2024-12_bronze      | In: 3,668,371 | Out: 3,563,400 | Removed:  2.86% | Clean: 97.93% | 102.1s
--------------------------------------------------------------------------------

BATCH PROCESSING COMPLETE!


In [12]:
# Generate Summary Report with Enrichment
print("=" * 80)
print("ENRICHED SILVER LAYER - FINAL SUMMARY REPORT")
print("=" * 80)
print()

print("OVERALL STATISTICS:")
print("-" * 80)
print(f"Files Processed:           {overall_stats['files_processed']}/24")
print(f"Total Input Records:       {overall_stats['total_input_records']:,}")
print(f"Total Records Removed:     {overall_stats['total_removed_records']:,} "
      f"({overall_stats['total_removed_records']/overall_stats['total_input_records']*100:.2f}%)")
print(f"Total Output Records:      {overall_stats['total_output_records']:,} "
      f"({overall_stats['total_output_records']/overall_stats['total_input_records']*100:.2f}%)")
print()

print("SILVER LAYER QUALITY DISTRIBUTION:")
print("-" * 80)
clean_pct = (overall_stats['total_clean_records']/overall_stats['total_output_records']*100)
flagged_pct = (overall_stats['total_flagged_records']/overall_stats['total_output_records']*100)
print(f"Clean Records:             {overall_stats['total_clean_records']:,} ({clean_pct:.2f}%)")
print(f"Flagged Records:           {overall_stats['total_flagged_records']:,} ({flagged_pct:.2f}%)")
print()

print("PERFORMANCE METRICS:")
print("-" * 80)
total_time = sum(overall_stats['processing_times'])
avg_time = total_time / len(overall_stats['processing_times'])
throughput = overall_stats['total_input_records'] / total_time

print(f"Total Processing Time:     {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
print(f"Average Time per File:     {avg_time:.1f} seconds")
print(f"Throughput:                {throughput:,.0f} records/second")
print()

print("DATA QUALITY TRENDS BY TIME PERIOD:")
print("-" * 80)
print(f"{'Period':<10} | {'Files':<5} | {'Removal %':<10} | {'Clean %':<10}")
print("-" * 80)

# Group by year
stats_2023 = [s for s in overall_stats['file_stats'] if '2023' in s['file']]
stats_2024 = [s for s in overall_stats['file_stats'] if '2024' in s['file']]

for period, stats in [('2023', stats_2023), ('2024', stats_2024)]:
    total_in = sum(s['input'] for s in stats)
    total_removed = sum(s['removed'] for s in stats)
    total_out = sum(s['output'] for s in stats)
    total_clean = sum(s['clean'] for s in stats)
    
    removal_pct = (total_removed / total_in * 100) if total_in > 0 else 0
    clean_pct = (total_clean / total_out * 100) if total_out > 0 else 0
    
    print(f"{period:<10} | {len(stats):>5} | {removal_pct:>9.2f}% | {clean_pct:>9.2f}%")

print()

print("ENRICHED SILVER LAYER SCHEMA:")
print("-" * 80)
print(f"Total Fields:              37")
print(f"  • Original Taxi Fields:  19")
print(f"  • Enrichment Fields:     9  ← NEW")
print(f"  • Bronze Metadata:       5")
print(f"  • Silver Metadata:       4")
print()
print("Enrichment Fields Added:")
print("  Location Enrichment (6):")
print("    - pickup_borough, pickup_zone, pickup_service_zone")
print("    - dropoff_borough, dropoff_zone, dropoff_service_zone")
print("  ID Enrichment (3):")
print("    - vendor_name, rate_description, payment_method")
print()

print("RDD MAPREDUCE OPERATIONS APPLIED:")
print("-" * 80)
print("Per Record Transformations:")
print("  1. map(row.asDict) - Convert Spark Rows to dictionaries")
print("  2. map(normalize_record) - Schema standardization")
print("  3. map(enrich_record) - Add human-readable fields ← NEW")
print("  4. map(validate_record_quality) - Quality validation")
print("  5. filter(!should_remove_record) - Remove critical issues")
print("  6. map(create_silver_record) - Add Silver metadata")
print("  7. map(dict_to_tuple) - Convert to tuple for DataFrame")
print()
print("Aggregation Operations:")
print("  • reduceByKey(sum) - Count records by status")
print("  • count() - Trigger RDD computation")
print()

print("STORAGE INFORMATION:")
print("-" * 80)

# Calculate directory sizes
bronze_size = sum(os.path.getsize(os.path.join(bronze_path, d, f)) 
                 for d in bronze_dirs 
                 for f in os.listdir(os.path.join(bronze_path, d)) 
                 if os.path.isfile(os.path.join(bronze_path, d, f)))

silver_dirs = [d.replace('_bronze', '_silver') for d in bronze_dirs]
silver_size = sum(os.path.getsize(os.path.join(silver_output_path, d, f)) 
                 for d in silver_dirs 
                 for f in os.listdir(os.path.join(silver_output_path, d)) 
                 if os.path.isfile(os.path.join(silver_output_path, d, f)))

print(f"Bronze Layer Size:         {bronze_size/(1024**3):.2f} GB")
print(f"Silver Layer Size:         {silver_size/(1024**3):.2f} GB")
print(f"Size Change:               {(silver_size/bronze_size-1)*100:+.1f}%")
print(f"  (Size increase due to 9 enrichment + 4 Silver metadata fields)")
print()

print("OUTPUT LOCATION:")
print("-" * 80)
print(f"Silver Layer: {silver_output_path}")
print(f"  • 24 monthly partitions")
print(f"  • Parquet format with Snappy compression")
print(f"  • 37 fields per record (enriched and cleaned)")
print()

print("=" * 80)
print("✓ ENRICHED SILVER LAYER COMPLETE!")
print("✓ All processing done using RDD MapReduce operations only")
print("✓ Data is cleaned, enriched, and ready for Gold layer analytics")
print("=" * 80)

ENRICHED SILVER LAYER - FINAL SUMMARY REPORT

OVERALL STATISTICS:
--------------------------------------------------------------------------------
Files Processed:           24/24
Total Input Records:       79,479,946
Total Records Removed:     2,507,056 (3.15%)
Total Output Records:      76,972,890 (96.85%)

SILVER LAYER QUALITY DISTRIBUTION:
--------------------------------------------------------------------------------
Clean Records:             75,938,992 (98.66%)
Flagged Records:           1,033,898 (1.34%)

PERFORMANCE METRICS:
--------------------------------------------------------------------------------
Total Processing Time:     3300.6 seconds (55.0 minutes)
Average Time per File:     137.5 seconds
Throughput:                24,080 records/second

DATA QUALITY TRENDS BY TIME PERIOD:
--------------------------------------------------------------------------------
Period     | Files | Removal %  | Clean %   
--------------------------------------------------------------------

In [13]:
# Verify Enriched Silver Layer Output
print("=" * 80)
print("ENRICHED SILVER LAYER OUTPUT VERIFICATION")
print("=" * 80)
print()

# Load a sample Silver file
sample_silver_path = f"{silver_output_path}/yellow_tripdata_2024-12_silver"
sample_silver_df = spark.read.parquet(sample_silver_path)

print(f"Sample File: {sample_silver_path}")
print(f"Record Count: {sample_silver_df.count():,}")
print()

print("Schema (37 fields total):")
print("-" * 80)
for i, field in enumerate(sample_silver_df.schema.fields, 1):
    field_type = str(field.dataType).replace('Type', '').replace('()', '')
    category = ""
    if i <= 19:
        category = "Original"
    elif i <= 28:
        category = "Enrichment"
    elif i <= 33:
        category = "Bronze Meta"
    else:
        category = "Silver Meta"
    print(f"{i:2}. {field.name:<35} {field_type:<15} [{category}]")

print()
print("Sample Enriched Records:")
print("-" * 80)

# Show sample clean record with enrichment
clean_sample = sample_silver_df.filter(sample_silver_df._silver_status == 'clean').limit(1)
print("\n✓ CLEAN RECORD (showing enrichment):")
clean_sample.select(
    'tpep_pickup_datetime',
    'pulocationid', 'pickup_zone', 'pickup_borough',
    'dolocationid', 'dropoff_zone', 'dropoff_borough',
    'vendorid', 'vendor_name',
    'payment_type', 'payment_method',
    'fare_amount', 'total_amount',
    '_silver_status'
).show(1, truncate=False, vertical=True)

print()
print("=" * 80)
print("✓ Enriched Silver Layer Verification Complete!")
print("✓ All ID fields now have human-readable descriptions")
print("✓ Ready for Gold layer analytics with enriched data")
print("=" * 80)

ENRICHED SILVER LAYER OUTPUT VERIFICATION

Sample File: /home/ubuntu/project2/silver_layer/yellow_tripdata_2024-12_silver
Record Count: 3,563,400

Schema (37 fields total):
--------------------------------------------------------------------------------
 1. vendorid                            Long            [Original]
 2. tpep_pickup_datetime                Timestamp       [Original]
 3. tpep_dropoff_datetime               Timestamp       [Original]
 4. passenger_count                     Double          [Original]
 5. trip_distance                       Double          [Original]
 6. ratecodeid                          Double          [Original]
 7. store_and_fwd_flag                  String          [Original]
 8. pulocationid                        Long            [Original]
 9. dolocationid                        Long            [Original]
10. payment_type                        Long            [Original]
11. fare_amount                         Double          [Original]
12. extra

---

## Visual Verification: Sample Records from Different Files

Let's examine sample records from different time periods to verify data quality and enrichment.

In [ ]:
# Visual Check: Display Top 5 Rows from Different Files

import pandas as pd

# Select sample files from different periods
sample_files = [
    "/home/ubuntu/dat535-2025-group10/silver_layer/yellow_tripdata_2023-01_silver",
    "/home/ubuntu/dat535-2025-group10/silver_layer/yellow_tripdata_2023-06_silver",
    "/home/ubuntu/dat535-2025-group10/silver_layer/yellow_tripdata_2024-01_silver",
    "/home/ubuntu/dat535-2025-group10/silver_layer/yellow_tripdata_2024-06_silver",
    "/home/ubuntu/dat535-2025-group10/silver_layer/yellow_tripdata_2024-12_silver"
]

for file_path in sample_files:
    file_name = file_path.split("/")[-1]
    print("=" * 100)
    print(f"FILE: {file_name}")
    print("=" * 100)
    
    # Load and display top 5 rows
    df = spark.read.parquet(file_path)
    
    # Show record count and schema
    print(f"Total Records: {df.count():,}")
    print(f"Total Fields: {len(df.columns)}")
    print()
    
    # Convert to Pandas for better display (top 5 rows only)
    sample_pd = df.limit(5).toPandas()
    
    # Display key original fields
    print("KEY TRIP FIELDS:")
    print("-" * 100)
    key_fields = ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                  'passenger_count', 'trip_distance', 'pulocationid', 'dolocationid',
                  'ratecodeid', 'payment_type', 'fare_amount', 'total_amount']
    available_key_fields = [f for f in key_fields if f in sample_pd.columns]
    print(sample_pd[available_key_fields].to_string(index=False))
    print()
    
    # Display enrichment fields
    print("ENRICHMENT FIELDS:")
    print("-" * 100)
    enrichment_fields = ['vendor_name', 'pickup_borough', 'pickup_zone', 
                         'dropoff_borough', 'dropoff_zone', 
                         'rate_description', 'payment_method']
    available_enrichment_fields = [f for f in enrichment_fields if f in sample_pd.columns]
    print(sample_pd[available_enrichment_fields].to_string(index=False))
    print()
    
    # Display Silver metadata
    print("SILVER METADATA:")
    print("-" * 100)
    metadata_fields = ['_silver_status', '_silver_issue_count', '_silver_quality_issues']
    available_metadata_fields = [f for f in metadata_fields if f in sample_pd.columns]
    print(sample_pd[available_metadata_fields].to_string(index=False))
    print()
    print()

print("=" * 100)
print("✓ Visual verification complete!")
print("=" * 100)

FILE: yellow_tripdata_2023-01_silver
Total Records: 2,970,852
Total Fields: 37



KEY TRIP FIELDS:
----------------------------------------------------------------------------------------------------
 vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  pulocationid  dolocationid  ratecodeid  payment_type  fare_amount  total_amount
        2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0           0.97           161           141         1.0             2          9.3         14.30
        2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0           1.10            43           237         1.0             1          7.9         16.90
        2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0           2.51            48           238         1.0             1         14.9         34.90
        2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0           1.43           107            79         1.0             1         11.4         19.68
        2  2023-01-01 00:50:34   2023-01-01 01:02:52

KEY TRIP FIELDS:
----------------------------------------------------------------------------------------------------
 vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  pulocationid  dolocationid  ratecodeid  payment_type  fare_amount  total_amount
        1  2023-06-01 00:08:48   2023-06-01 00:29:41              1.0           3.40           140           238         1.0             1         21.9         33.60
        1  2023-06-01 00:48:24   2023-06-01 01:07:07              1.0          10.20           138            97         1.0             1         40.8         60.05
        2  2023-06-01 00:54:03   2023-06-01 01:17:29              3.0           9.83           100           244         1.0             1         39.4         53.28
        2  2023-06-01 00:18:44   2023-06-01 00:27:18              1.0           1.17           137           234         1.0             1          9.3         15.02
        1  2023-06-01 00:32:36   2023-06-01 00:45:52

In [ ]:
spark.stop()